# Detekcija etiketa boca

**Autor:** [fvucemilo](https://github.com/fvucemilo)<br>
**Datum kreiranja:** 27/07/2021<br>
**Zadnje uređivano:** 05/11/2021<br>
**Opis:** Obučavanje klasifikatora slika na skupu podataka Etikete boca vina.

## Uvod

Pomoćna skripta koja učitava slike boca obrađenih bez pozadine po kalasama. Nad njima pronalazi konture etiketa označava ih i sprema istim redosljedom u mapu sa etiketama koje se kasnije koriste za treniranje modela.  

In [ ]:
import os
import cv2
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import drive

## Google Drive konekcija

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Deklariranje konstanti

In [ ]:
# Deklariranje konstanti

# Putanja do mape koja sadrži slike bez pozadine
DATA_DIR = pathlib.Path('gdrive/MyDrive/BoceNB')

#Putanja do mape za spremanje slika etiketa
LABEL_DIR = pathlib.Path('gdrive/MyDrive/Etikete')

In [ ]:
for (subdir, dirs, files) in os.walk(DATA_DIR):
    for file in files:
        image = cv2.imread(os.path.join(subdir, file))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        (ret, thresh) = cv2.threshold(gray, 125, 255, cv2.THRESH_BINARY)

        (contours, hierarchy) = cv2.findContours(thresh,
                cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        if len(contours) != 0:
            # Iscrtavanje plavom bojom konture koje su utemeljene
            # cv2.drawContours(image, contours, -1, 255, 3)

            # Pronalaženje najveće konture (c) po površini
            c = max(contours, key=cv2.contourArea)
            (x, y, w, h) = cv2.boundingRect(c)

            # Iscrtavanje najveće konture (c) zelenom bojom
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 5)

        foreground = image[y:y + h, x:x + w]

        # Vizualiziranj obrade
        plt.figure(figsize=(10, 5))
        (plt.subplot(1, 3, 1), plt.imshow(image), plt.title('Ulaz'))
        (plt.subplot(1, 3, 2), plt.imshow(image), plt.title('Sve konture'))
        (plt.subplot(1, 3, 3), plt.imshow(foreground), plt.title('Izlaz'))
        plt.show()

        label_path = os.path.join(subdir)

        # Zamjena putanja učitanih slika sa izlazom putanjom za spremanje
        if label_path.startswith(str(DATA_DIR)):
            label_path = label_path.replace(str(DATA_DIR), str(LABEL_DIR))

        # Kreiranje direktorija na izlaznoj putanji ako ne postoje
        if os.path.exists(label_path) == False:
            os.makedirs(label_path)
            
        label_filename_path = os.path.join(label_path, file)

        # Spremanje slika etiketa na izlaznu putanju
        cv2.imwrite(label_filename_path, foreground)